##📦 Instalação e Importações

In [ ]:
# Instala pacotes necessários
%pip install sidrapy
%pip install openpyxl

# Importações essenciais
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from IPython.display import display
import sidrapy
from sidrapy import get_table
import sklearn
import os
import time
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##LOAD & TRANSFORM - SIDRA | TABELA 5457 (2022)

In [126]:
#Load na base de 2022
caminho_arquivo = '/content/drive/MyDrive/DNC/Projetos/Becomex/01. Base de dados/SIDRA/RAW/2022/(2022) SIDRA Top10 - 5457.xlsx'


In [127]:
# Carregar todas as abas
abas = pd.ExcelFile(caminho_arquivo)

# Ler a aba "Área plantada (Hectares)"
df_area_plantada = abas.parse("Área plantada (Hectares)")

# Verificar os tipos de dados por coluna
print(df_area_plantada.dtypes)

Nível                                                                                               object
Grande Região, Unidade da Federação, Município, Mesorregião Geográfica e Microrregião Geográfica    object
Total                                                                                               object
Algodão herbáceo (em caroço)                                                                        object
Arroz (em casca)                                                                                    object
Cacau (em amêndoa)                                                                                  object
Café (em grão) Total                                                                                object
Cana-de-açúcar                                                                                      object
Feijão (em grão)                                                                                    object
Fumo (em folha)                      

In [138]:
# Carregar a primeira aba do arquivo
df = pd.read_excel(caminho_arquivo, sheet_name=0)

# Corrigir tipo da coluna categórica
df['Nível'] = df['Nível'].astype('category')

# Listar as colunas numéricas
colunas_numericas = [
    'Total',
    'Algodão herbáceo (em caroço)',
    'Arroz (em casca)',
    'Cacau (em amêndoa)',
    'Café (em grão) Total',
    'Cana-de-açúcar',
    'Feijão (em grão)',
    'Fumo (em folha)',
    'Laranja',
    'Milho (em grão)',
    'Soja (em grão)'
]

# Converter colunas numéricas com segurança
df[colunas_numericas] = df[colunas_numericas].apply(pd.to_numeric, errors='coerce')

# Adicionar coluna de ano (série temporal)
df['Ano'] = 2022

# Verificar tipos finais
print(df.dtypes)


KeyboardInterrupt: 

In [130]:
# Visualizar as 10 primeiras linhas
df.head(10)

,Nível,"Grande Região, Unidade da Federação, Município, Mesorregião Geográfica e Microrregião Geográfica",Total,Algodão herbáceo (em caroço),Arroz (em casca),Cacau (em amêndoa),Café (em grão) Total,Cana-de-açúcar,Feijão (em grão),Fumo (em folha),Laranja,Milho (em grão),Soja (em grão),Ano
0,GR,Norte,5550519.0,15535.0,208353.0,161704.0,59044.0,53624.0,98945.0,152.0,19407.0,1211328.0,2617967.0,2022
1,GR,Nordeste,11916290.0,348902.0,160807.0,424745.0,118777.0,925905.0,1313314.0,21450.0,94688.0,2934118.0,3753375.0,2022
2,GR,Sudeste,16053282.0,36034.0,14681.0,15627.0,1641783.0,6592568.0,386869.0,408.0,403419.0,2165005.0,3275074.0,2022
3,GR,Sul,23786770.0,48.0,1129388.0,NaN,30773.0,538086.0,605445.0,306878.0,43910.0,4194084.0,12825384.0,2022
4,GR,Centro-Oeste,33994538.0,1249105.0,143179.0,724.0,19190.0,1794130.0,328937.0,120.0,9024.0,10805714.0,18825863.0,2022
5,UF,Rondônia,961956.0,10050.0,40221.0,7563.0,57475.0,502.0,3137.0,NaN,373.0,322237.0,489526.0,2022
6,UF,Acre,92287.0,NaN,3705.0,NaN,1062.0,480.0,5235.0,142.0,493.0,40460.0,6570.0,2022
7,UF,Amazonas,120030.0,NaN,1065.0,1214.0,320.0,4775.0,989.0,NaN,1237.0,2554.0,5900.0,2022
8,UF,Roraima,150608.0,NaN,10710.0,90.0,NaN,190.0,965.0,NaN,1435.0,15720.0,102785.0,2022
9,UF,Pará,2313220.0,NaN,40556.0,152837.0,187.0,17479.0,27360.0,10.0,15197.0,405590.0,839560.0,2022


In [133]:
# 📄 Listar as abas do Excel
abas = pd.ExcelFile(caminho_arquivo).sheet_names

# 🏷️ Colunas categóricas e numéricas
coluna_categoria = 'Nível'
colunas_numericas = [
    'Total',
    'Algodão herbáceo (em caroço)',
    'Arroz (em casca)',
    'Cacau (em amêndoa)',
    'Café (em grão) Total',
    'Cana-de-açúcar',
    'Feijão (em grão)',
    'Fumo (em folha)',
    'Laranja',
    'Milho (em grão)',
    'Soja (em grão)'
]

# 📦 Dicionário para armazenar os DataFrames
dfs = {}

# 🔁 Loop pelas abas do Excel
for nome_aba in abas:
    # Nome do DataFrame com ano
    nome_df = f"df_2022_{nome_aba.lower().replace(' ', '_').replace('-', '_').replace('(', '').replace(')', '')}"

    # 📥 Ler aba
    df_temp = pd.read_excel(caminho_arquivo, sheet_name=nome_aba)

    # 🏷️ Tipagem da coluna categórica
    if coluna_categoria in df_temp.columns:
        df_temp[coluna_categoria] = df_temp[coluna_categoria].astype('category')

    # 🔢 Tipagem de colunas numéricas
    for col in colunas_numericas:
        if col in df_temp.columns:
            df_temp[col] = pd.to_numeric(df_temp[col], errors='coerce')

    # 📆 Adicionar coluna de ano
    df_temp['Ano'] = 2022

    # 💾 Salvar no dicionário
    dfs[nome_df] = df_temp


In [134]:
dfs.keys()


dict_keys(['df_2022_área_plantada_hectares', 'df_2022_área_percentual_do_total_geral', 'df_2022_área_colhida_hectares', 'df_2022_área_colhida___percentual_total', 'df_2022_qtd_produzida___toneladas', 'df_2022_rendimento_médio_kg_x_hectare', 'df_2022_valor_da_produção_mil_reais', 'df_2022_valor_da_produção___%_do_total_'])

In [135]:
# 📦 Dicionário final para armazenar os DataFrames por aba e nível
dfs_nivel = {}

# 🔁 Iterar sobre os DataFrames já carregados e tipados
for nome_df, df in dfs.items():

    # ✅ Garantir que a coluna 'Nível' exista e seja categórica
    if 'Nível' in df.columns:
        df['Nível'] = df['Nível'].astype('category')

        # 🕒 Extrair o ano a partir do nome do DataFrame
        if "2022" in nome_df:
            ano = 2022
        elif "2023" in nome_df:
            ano = 2023
        else:
            ano = None  # fallback para debugging se necessário

        # 🏷️ Adicionar coluna 'Ano' ao DataFrame
        df['Ano'] = ano

        # 🔢 Colunas numéricas a serem convertidas
        colunas_numericas = [
            'Total',
            'Algodão herbáceo (em caroço)',
            'Arroz (em casca)',
            'Cacau (em amêndoa)',
            'Café (em grão) Total',
            'Cana-de-açúcar',
            'Feijão (em grão)',
            'Fumo (em folha)',
            'Laranja',
            'Milho (em grão)',
            'Soja (em grão)'
        ]

        for col in colunas_numericas:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # 🔍 Separar e salvar por cada nível único encontrado
        for nivel in df['Nível'].dropna().unique():
            nivel_nome = str(nivel).strip().lower().replace(" ", "_")
            nome_final = f"{nome_df}_{nivel_nome}"

            # Criar cópia do DataFrame filtrado
            df_nivel = df[df['Nível'] == nivel].copy()

            # Armazenar no dicionário
            dfs_nivel[nome_final] = df_nivel

# ✅ Visualizar os primeiros nomes gerados
for nome in list(dfs_nivel.keys())[:10]:
    print(nome)


df_2022_área_plantada_hectares_gr
df_2022_área_plantada_hectares_uf
df_2022_área_plantada_hectares_mu
df_2022_área_plantada_hectares_me
df_2022_área_plantada_hectares_mi
df_2022_área_plantada_hectares_fonte:_ibge_-_produção_agrícola_municipal
df_2022_área_percentual_do_total_geral_gr
df_2022_área_percentual_do_total_geral_uf
df_2022_área_percentual_do_total_geral_mu
df_2022_área_percentual_do_total_geral_me


In [136]:
# Caminho da pasta já existente
caminho_destino = '/content/drive/MyDrive/DNC/Projetos/Becomex/01. Base de dados/SIDRA/TRATADOS/2022/'

# Salvar cada DataFrame do dicionário dfs_nivel
for nome, df in dfs_nivel.items():
    df.to_excel(f"{caminho_destino}{nome}.xlsx", index=False)

##LOAD & TRANSFORM - SIDRA | TABELA 5457 (2023)

In [147]:
import pandas as pd
import os

# Caminhos
caminho_arquivo_2023 = '/content/drive/MyDrive/DNC/Projetos/Becomex/01. Base de dados/SIDRA/RAW/2023/(2023) -  SIDRA Top10 - 5457.xlsx'
caminho_saida = '/content/drive/MyDrive/DNC/Projetos/Becomex/01. Base de dados/SIDRA/TRATADOS/2023'

# Parâmetros
coluna_categoria = 'Nível'
colunas_numericas = [
    'Algodão herbáceo (em caroço)',
    'Arroz (em casca)',
    'Cacau (em amêndoa)',
    'Café (em grão) Total',
    'Cana-de-açúcar',
    'Feijão (em grão)',
    'Fumo (em folha)',
    'Laranja',
    'Milho (em grão)',
    'Soja (em grão)'
]
ano_referencia = 2023

# Listar abas
abas_2023 = pd.ExcelFile(caminho_arquivo_2023).sheet_names
dfs_2023 = {}

# Carregar e tratar cada aba
for nome_aba in abas_2023:
    nome_base = nome_aba.lower().replace(" ", "_").replace("-", "_").replace("(", "").replace(")", "")
    nome_df = f"df_2023_{nome_base}"
    df_temp = pd.read_excel(caminho_arquivo_2023, sheet_name=nome_aba)

    # Tipagem
    if coluna_categoria in df_temp.columns:
        df_temp[coluna_categoria] = df_temp[coluna_categoria].astype('category')

    for col in colunas_numericas:
        if col in df_temp.columns:
            df_temp[col] = pd.to_numeric(df_temp[col], errors='coerce')

    # Criar coluna Total
    colunas_validas = [col for col in colunas_numericas if col in df_temp.columns]
    df_temp['Total'] = df_temp[colunas_validas].sum(axis=1)

    # Criar coluna Ano
    df_temp['Ano'] = ano_referencia

    # Reorganizar colunas: Nível, Local, Total, Métricas..., Ano
    colunas = list(df_temp.columns)
    geo_col = colunas[1]
    nova_ordem = [coluna_categoria, geo_col, 'Total'] + [c for c in colunas if c not in [coluna_categoria, geo_col, 'Total', 'Ano']] + ['Ano']
    df_temp = df_temp[nova_ordem]

    dfs_2023[nome_df] = df_temp

# Separar por nível e exportar
for nome_df, df in dfs_2023.items():
    if 'Nível' in df.columns:
        for nivel in df['Nível'].dropna().unique():
            nivel_nome = str(nivel).strip().lower().replace(" ", "_")
            nome_final = f"{nome_df}_{nivel_nome}"

            df_nivel = df[df['Nível'] == nivel].copy()

            # Garantir ordenação correta
            colunas = list(df_nivel.columns)
            if 'Total' in colunas:
                colunas.remove('Total')
                colunas.insert(2, 'Total')
            if 'Ano' in colunas:
                colunas.remove('Ano')
                colunas.append('Ano')
            df_nivel = df_nivel[colunas]

            # Exportar
            caminho_saida_final = os.path.join(caminho_saida, f"{nome_final}.xlsx")
            df_nivel.to_excel(caminho_saida_final, index=False)
